# HMAC Generator and Checker Example

This notebook demonstrates how to use the `HMACGenerator` and `HMACCheck` classes to generate and verify HMACs for packets within a page. We will walk through creating packets, generating HMACs, and verifying their integrity.

## Step 1: Import Required Classes

Let's start by importing the necessary classes and libraries.


In [1]:
import numpy as np
import sys
sys.path.append('../')
from Book import Packet, Page
from src import MACGenerator, MACChecker

## Step 2: Create a Page with Packets

We will create a `Page` object that holds multiple `Packet` instances. Each packet will have a unique message.


In [2]:
# Initialize a sample page with packets
page = Page(page_size=2)
page.add_packet(Packet(SN=0, message=b"Message 1"))
page.add_packet(Packet(SN=1, message=b"Message 2"))

# Display the packets in the page
for packet in page.packets:
    print(f"Packet SN: {packet.SN}, Message: {packet.message}, MAC: {packet.mac}")


Packet SN: 0, Message: b'Message 1', MAC: b''
Packet SN: 1, Message: b'Message 2', MAC: b''


## Step 3: Define Matrices X and Y

We define matrices `X` and `Y` to specify how the HMACs should be generated and where they should be placed within the packets.


In [3]:
# Example matrices X and Y
X = np.array([[1, 1],  # t1 uses messages from p1 and p2
              [0, 1]], dtype=int)  # t2 uses message from p2 only

Y = np.array([[0, 1],  # t1 is placed in p2's MAC
              [1, 0]], dtype=int)  # t2 is placed in p1's MAC

# Display matrices X and Y
print("Matrix X:\n", X)
print("Matrix Y:\n", Y)

Matrix X:
 [[1 1]
 [0 1]]
Matrix Y:
 [[0 1]
 [1 0]]


## Step 4: Generate HMACs for the Packets

Using the `HMACGenerator`, we will generate HMACs for the packets based on the matrices `X` and `Y`.


In [4]:
# Initialize the HMAC generator with a secret key
hmac_generator = MACGenerator(X = X, Y = Y, secret_key=b'secret')

# Apply HMACs to the page
hmac_generator.process_page(page)

# Display the packets after applying HMACs
for packet in page.packets:
    print(f"Packet SN: {packet.SN}, Message: {packet.message}, MAC: {packet.mac}")

Packet SN: 0, Message: b'Message 1', MAC: b'k\xe80\x92\xbd9d\xb8\xb2\x03N\xc9\n\x84\xbb\x02D\xd6\x83+mX\xbe8T`9\xfe\xb2s/\x84'
Packet SN: 1, Message: b'Message 2', MAC: b'\xc3\xd2\xbe\x83\x0fpq\xba\x04\xb4\x12)\xff\xf7\xaab\x7f\xbf[-\x8e\x1a\x1d\x00\xa3\x17\xc7\xdbNP\x1c&'


## Step 5: Verify the HMACs

Now, we will use the `HMACCheck` class to verify that the HMACs in the packets are correct.


In [5]:
# Initialize the HMAC checker with the same secret key
hmac_checker = MACChecker(X= X, Y = Y, secret_key=b'secret')

# Check if the HMACs in the page are valid
is_valid = hmac_checker.check_page(page)

print(f"Are all HMACs valid? {is_valid}")


Are all HMACs valid? (b'Message 1Message 2', array([2, 1]), array([0.01312399, 0.01309943]))


## Step 6: Test HMAC Verification with a Tampered Packet

We will manually tamper with one of the packet's MACs to simulate an integrity breach and verify that the HMACCheck class detects the issue.


In [6]:
# Tamper with one of the MACs to simulate an invalid HMAC
page.packets[0].mac = b'Invalid MAC'
# Check if the HMACs in the page are still valid
is_valid_after_tampering = hmac_checker.check_page(page)

print(f"Are all HMACs valid after tampering? {is_valid_after_tampering}")

Are all HMACs valid after tampering? (b'Message 1', array([1, 0]), array([0.01738739, 0.01736283]))


## Summary

In this notebook, we demonstrated how to:
- Create packets and store them in a page.
- Generate HMACs for the packets based on specified matrices.
- Verify the HMACs to ensure the integrity of the packets.
- Detect tampering by checking HMAC validity after altering a packet.

This process is crucial in ensuring data integrity when transmitting data over untrusted networks.
